In [1]:
from pathlib import Path

from model_client import ModelClient, get_prompt
from pdf_extraction import extract_text_from_pdf

In [7]:
pdf_path = Path("./data/pdfs")
client = ModelClient()

responses = []
for pdf in list(pdf_path.glob("*.pdf"))[:2]:
    print(pdf)
    paper_text = extract_text_from_pdf(pdf)
    prompt = get_prompt(paper_text)
    response = client.ask_llama(prompt)
    responses.append(response)

data/pdfs/pdf (2).pdf


KeyboardInterrupt: 

In [ ]:
responses

'Here are the evaluations for each paper:\n\n1. {Paper Title: "Biosensors 2022, 12, 202 of 19 of 19"}\n   Decision: Exclude\n   Reason(s): The paper title does not meet the inclusion criteria as it is not a specific research article but rather a general journal entry.\n   Note: No relevant quotes from the paper are provided.\n\n2. {Paper Title: "The discretewavelettransform: Weddingtheatrous andMallatalgorithms."}\n   Decision: Exclude\n   Reason(s): The paper title does not meet the inclusion criteria as it is an older, non-technical journal article (published in 1992).\n   Note: No relevant quotes from the paper are provided.\n\n3. {Paper Title: "The Normal Electrocardiogram."}\n   Decision: Include\n   Reason(s): The paper provides a classic reference on normal electrocardiograms, making it a valuable resource for researchers.\n   Note: None\n\n4. {Paper Title: "Elevated resting heart rate is an independent risk factor for cardiovascular disease in healthy men and women."}\n   Decis